In [ ]:
# %% [markdown]
# # Сегментация клиентов банка с использованием GMM
#
# Кластеризация клиентов на основе `DECENTRATHON_3.0.parquet` с использованием Gaussian Mixture Model (GMM, k=7).
# Реализованы сегменты (Понижающийся, Повышающийся, Стабильный, Остановившийся) и атрибуты (Geo, Behavior, Category).
# Зависимости устанавливаются автоматически без перезапуска runtime.
# Результаты сохраняются в `segmented.parquet` с процентами сегментов и атрибутов.
#
# **Текущая дата и время**: 11:33:00 AM +05, Sunday, May 25, 2025
#
# **Запуск в Google Colab**:
# - Загрузите `DECENTRATHON_3.0.parquet` в `/content/` или Google Drive.
# - Выполняйте ячейки последовательно.

# %% [markdown]
# ## 0. Установка зависимостей

import pkg_resources
import subprocess
import sys
import os

requirements = """
pandas==2.2.2
numpy==1.26.4
scikit-learn==1.5.1
matplotlib==3.9.2
seaborn==0.13.2
joblib==1.4.2
onnx==1.16.2
skl2onnx==1.16.0
faker==28.1.0
pyarrow==17.0.0
"""

with open('/content/requirements.txt', 'w') as f:
    f.write(requirements)

def install_requirements():
    print("Проверка зависимостей...")
    installed = {pkg.key: pkg.version for pkg in pkg_resources.working_set}
    for line in requirements.strip().split('\n'):
        package = line.strip()
        if not package:
            continue
        pkg_name, pkg_version = package.split('==')
        pkg_name = pkg_name.lower()
        if pkg_name not in installed or installed[pkg_name] != pkg_version:
            print(f"Установка {package}...")
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', package, '--quiet'])
        else:
            print(f"{package} уже установлен")

try:
    install_requirements()
    print("Все зависимости готовы")
except Exception as e:
    print(f"Ошибка установки зависимостей: {e}")
    raise

# %% [markdown]
# ## 1. Импорт библиотек

import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import logging
from datetime import datetime, timedelta
from faker import Faker
import warnings
warnings.filterwarnings('ignore')

logging.basicConfig(filename="/content/segmentation.log", level=logging.INFO)
faker = Faker('ru_RU')
np.random.seed(42)
current_datetime = datetime(2025, 5, 25, 11, 33, 0)
print(f"Текущая дата и время: {current_datetime.strftime('%I:%M:%S %p +05, %A, %B %d, %Y')}")

# %% [markdown]
# ## 2. Загрузка данных

from google.colab import drive

try:
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive подключён")
except Exception as e:
    print(f"Ошибка подключения Google Drive: {e}")

file_path = '/content/drive/MyDrive/DECENTRATHON_3.0.parquet'
alternative_path = '/content/DECENTRATHON_3.0.parquet'

try:
    if os.path.exists(file_path):
        df = pd.read_parquet(file_path, columns=[
            'card_id', 'transaction_timestamp', 'transaction_amount_kzt', 'merchant_mcc',
            'transaction_type', 'acquirer_country_iso', 'merchant_city', 'wallet_type',
            'issuer_bank_name'
        ])
        print("Данные загружены из", file_path)
    elif os.path.exists(alternative_path):
        df = pd.read_parquet(alternative_path, columns=[
            'card_id', 'transaction_timestamp', 'transaction_amount_kzt', 'merchant_mcc',
            'transaction_type', 'acquirer_country_iso', 'merchant_city', 'wallet_type',
            'issuer_bank_name'
        ])
        print("Данные загружены из", alternative_path)
    else:
        raise FileNotFoundError("Файл не найден")

    print(f"Размер датасета: {df.shape}")
    print("\nПервые 5 строк:")
    display(df.head())
except FileNotFoundError:
    print("Файл не найден. Проверьте пути:")
    print("- Google Drive:", file_path)
    print("- Локально:", alternative_path)
    print("Действия:")
    print("1. Загрузите файл в Google Drive: /MyDrive/DECENTRATHON_3.0.parquet")
    print("2. Или перетащите файл в Colab: /content/DECENTRATHON_3.0.parquet")
    print("3. Обновите file_path, если файл в другой папке")
    print("4. Для тестирования сгенерируйте тестовый датасет:")
    print("""
    from faker import Faker
    faker = Faker('ru_RU')
    test_data = [{
        'card_id': i,
        'transaction_timestamp': faker.date_time_between(start_date='-365d', end_date='now'),
        'transaction_amount_kzt': faker.random_int(1000, 100000),
        'merchant_mcc': faker.random_element([3000, 5812, 5611, 7994]),
        'transaction_type': faker.random_element(['POS', 'WEB', 'P2P']),
        'acquirer_country_iso': faker.random_element(['KAZ', 'USA']),
        'merchant_city': faker.city(),
        'wallet_type': faker.random_element([None, 'Apple Pay']),
        'issuer_bank_name': faker.company()
    } for i in range(1000)]
    df = pd.DataFrame(test_data)
    df.to_parquet('/content/DECENTRATHON_3.0.parquet')
    file_path = '/content/DECENTRATHON_3.0.parquet'
    """)
    raise
except Exception as e:
    print(f"Ошибка загрузки данных: {e}")
    raise

# %% [markdown]
# ## 3. Подготовка фичей

metrics = []
for card_id, group in df.groupby('card_id'):
    try:
        last_tx = group['transaction_timestamp'].max()
        first_tx = group['transaction_timestamp'].min()
        days_365_ago = current_datetime - timedelta(days=365)
        days_90_ago = current_datetime - timedelta(days=90)
        days_30_ago = current_datetime - timedelta(days=30)
        days_60_ago = current_datetime - timedelta(days=60)

        days_since_first_tx = (current_datetime - first_tx).days if pd.notnull(first_tx) else 0
        recency_days = (current_datetime - last_tx).days if pd.notnull(last_tx) else 365
        recent_tx = group[group['transaction_timestamp'] >= days_90_ago]
        frequency_90d = len(recent_tx)
        monetary_90d_kzt = recent_tx['transaction_amount_kzt'].sum() if not recent_tx.empty else 0
        average_check_kzt = monetary_90d_kzt / frequency_90d if frequency_90d > 0 else 0

        tx_0_30 = len(group[group['transaction_timestamp'] >= days_30_ago])
        tx_30_60 = len(group[(group['transaction_timestamp'] >= days_60_ago) & 
                             (group['transaction_timestamp'] < days_30_ago)])
        delta_count_30d = tx_0_30 - tx_30_60
        sum_0_30 = group[group['transaction_timestamp'] >= days_30_ago]['transaction_amount_kzt'].sum() if not group[group['transaction_timestamp'] >= days_30_ago].empty else 0
        sum_30_60 = group[(group['transaction_timestamp'] >= days_60_ago) & 
                          (group['transaction_timestamp'] < days_30_ago)]['transaction_amount_kzt'].sum() if not group[(group['transaction_timestamp'] >= days_60_ago) & (group['transaction_timestamp'] < days_30_ago)].empty else 0
        delta_sum_30d = sum_0_30 - sum_30_60
        avg_check_0_30 = sum_0_30 / tx_0_30 if tx_0_30 > 0 else 0
        avg_check_30_60 = sum_30_60 / tx_30_60 if tx_30_60 > 0 else 0
        delta_avg_check_30d = avg_check_0_30 - avg_check_30_60 if (tx_0_30 > 0 or tx_30_60 > 0) else 0

        total_amount = group['transaction_amount_kzt'].sum() if not group.empty else 0
        mcc_share_travel = group[group['merchant_mcc'].between(3000, 3299)]['transaction_amount_kzt'].sum() / total_amount if total_amount > 0 else 0
        mcc_share_entertainment = group[group['merchant_mcc'].isin([5812, 5814])]['transaction_amount_kzt'].sum() / total_amount if total_amount > 0 else 0
        mcc_share_fashion = group[group['merchant_mcc'].isin([5611, 5621, 5631, 5641, 5651, 5661, 5691, 5699])]['transaction_amount_kzt'].sum() / total_amount if total_amount > 0 else 0
        mcc_share_home = group[group['merchant_mcc'].isin([5200, 5211, 5231, 5251])]['transaction_amount_kzt'].sum() / total_amount if total_amount > 0 else 0
        mcc_share_tech = group[group['merchant_mcc'].isin([5732, 5733, 5734])]['transaction_amount_kzt'].sum() / total_amount if total_amount > 0 else 0
        mcc_share_gaming = group[group['merchant_mcc'] == 7994]['transaction_amount_kzt'].sum() / total_amount if total_amount > 0 else 0

        p2p_share = len(group[group['transaction_type'] == 'P2P']) / len(group) if len(group) > 0 else 0
        acquirer_country_iso_share = len(group[group['acquirer_country_iso'] == 'KAZ']) / len(group) if len(group) > 0 else 0
        merchant_city_unique = group['merchant_city'].nunique()
        total_amount_365d = group[group['transaction_timestamp'] >= days_365_ago]['transaction_amount_kzt'].sum() if not group[group['transaction_timestamp'] >= days_365_ago].empty else 0
        foreign_amount_365d = group[(group['transaction_timestamp'] >= days_365_ago) & 
                                    (group['acquirer_country_iso'] != 'KAZ')]['transaction_amount_kzt'].sum() if not group[(group['transaction_timestamp'] >= days_365_ago) & (group['acquirer_country_iso'] != 'KAZ')].empty else 0
        foreign_share_365d = foreign_amount_365d / total_amount_365d if total_amount_365d > 0 else 0

        if abs(monetary_90d_kzt - recent_tx['transaction_amount_kzt'].sum()) > 1e-6:
            logging.warning(f"Несоответствие monetary_90d_kzt для card_id {card_id}")

        metrics.append({
            'card_id': card_id,
            'days_since_first_tx': days_since_first_tx,
            'recency_days': recency_days,
            'frequency_90d': frequency_90d,
            'monetary_90d_kzt': monetary_90d_kzt,
            'average_check_kzt': average_check_kzt,
            'delta_count_30d': delta_count_30d,
            'delta_sum_30d': delta_sum_30d,
            'delta_avg_check_30d': delta_avg_check_30d,
            'sum_0_30': sum_0_30,
            'sum_30_60': sum_30_60,
            'avg_check_0_30': avg_check_0_30,
            'avg_check_30_60': avg_check_30_60,
            'mcc_share_travel': mcc_share_travel,
            'mcc_share_entertainment': mcc_share_entertainment,
            'mcc_share_fashion': mcc_share_fashion,
            'mcc_share_home': mcc_share_home,
            'mcc_share_tech': mcc_share_tech,
            'mcc_share_gaming': mcc_share_gaming,
            'p2p_share': p2p_share,
            'acquirer_country_iso_share': acquirer_country_iso_share,
            'merchant_city_unique': merchant_city_unique,
            'foreign_share_365d': foreign_share_365d,
            'issuer_bank_name': group['issuer_bank_name'].iloc[0] if not group['issuer_bank_name'].empty else "Неизвестно"
        })
    except Exception as e:
        logging.error(f"Ошибка обработки card_id {card_id}: {e}")
        continue

metrics_df = pd.DataFrame(metrics)
metrics_df['client_name'] = [faker.name() for _ in range(len(metrics_df))]
metrics_df.to_csv('/content/metrics.csv', index=False)
print("Метрики сохранены в /content/metrics.csv")
print(f"Размер метрик: {metrics_df.shape}")
print("\nПервые 5 строк метрик:")
display(metrics_df.head())

# %% [markdown]
# ## 4. Присвоение атрибутов и сегментов

def assign_attributes_and_segment(row):
    geo_attr, behavior_attr, category_attr, segment_attr = None, None, None, None
    geo_criteria, behavior_criteria, category_criteria, segment_criteria = [], [], [], []
    
    try:
        # Geo Attributes
        if row['acquirer_country_iso_share'] == 1.0:
            geo_attr = "Национальный"
            geo_criteria.append("acquirer_country_iso_share == 1.0")
        elif row['foreign_share_365d'] >= 0.01:
            geo_attr = "Интернациональный"
            geo_criteria.append("foreign_share_365d >= 0.01")
        elif row['merchant_city_unique'] == 1:
            geo_attr = "Местный"
            geo_criteria.append("merchant_city_unique == 1")
        else:
            geo_attr = "Неопределённый"
            geo_criteria.append("Не соответствует критериям geo")

        # Behavior Attributes
        # Примечание: Нет данных о кредитах/просрочках. Реализовано только на основе P2P и гейминга.
        # Если есть датасет с кредитами (например, late_payments_count), добавьте:
        # late_payments = row['late_payments_count'] > 0  # Пример
        if row['mcc_share_gaming'] > 0 or row['p2p_share'] > 0.3:
            behavior_attr = "Рискованный"
            behavior_criteria.append("mcc_share_gaming > 0 or p2p_share > 0.3")
        elif (row['recency_days'] <= 30 and row['delta_avg_check_30d'] >= 0 and 
              row['p2p_share'] == 0):
            behavior_attr = "Звездный"
            behavior_criteria.append("recency_days <= 30 and delta_avg_check_30d >= 0 and p2p_share == 0")
        else:
            behavior_attr = "Стандартный"
            behavior_criteria.append("Не соответствует критериям behavior")

        # Category Attributes
        if row['mcc_share_travel'] >= 0.3:
            category_attr = "Любитель путешествий"
            category_criteria.append("mcc_share_travel >= 0.3")
        elif row['mcc_share_entertainment'] >= 0.3:
            category_attr = "Гастроном"
            category_criteria.append("mcc_share_entertainment >= 0.3")
        elif row['mcc_share_fashion'] >= 0.3:
            category_attr = "Модник"
            category_criteria.append("mcc_share_fashion >= 0.3")
        elif row['mcc_share_home'] >= 0.3:
            category_attr = "Домосед"
            category_criteria.append("mcc_share_home >= 0.3")
        elif row['mcc_share_tech'] >= 0.3:
            category_attr = "Техногик"
            category_criteria.append("mcc_share_tech >= 0.3")
        elif row['mcc_share_gaming'] >= 0.3:
            category_attr = "Геймер"
            category_criteria.append("mcc_share_gaming >= 0.3")
        else:
            category_attr = "Стандартный"
            category_criteria.append("Все mcc_share < 0.3")

        # Segments
        if row['avg_check_0_30'] == 0:
            segment_attr = "Остановившийся"
            segment_criteria.append("avg_check_0_30 == 0")
        elif row['avg_check_0_30'] < row['avg_check_30_60'] and row['avg_check_30_60'] > 0:
            segment_attr = "Понижающийся"
            segment_criteria.append("avg_check_0_30 < avg_check_30_60")
        elif row['sum_0_30'] > row['sum_30_60']:
            segment_attr = "Повышающийся"
            segment_criteria.append("sum_0_30 > sum_30_60")
        elif row['sum_30_60'] > 0 and abs(row['sum_0_30'] - row['sum_30_60']) / row['sum_30_60'] < 0.1:
            segment_attr = "Стабильный"
            segment_criteria.append("abs(sum_0_30 - sum_30_60) / sum_30_60 < 0.1")
        else:
            segment_attr = "Неопределённый"
            segment_criteria.append("Не соответствует критериям сегмента")

    except Exception as e:
        geo_criteria.append(f"Ошибка в geo: {e}")
        behavior_criteria.append(f"Ошибка в behavior: {e}")
        category_criteria.append(f"Ошибка в category: {e}")
        segment_criteria.append(f"Ошибка в segment: {e}")
        geo_attr = behavior_attr = category_attr = segment_attr = "Неопределённый"

    return (geo_attr, behavior_attr, category_attr, segment_attr,
            geo_criteria, behavior_criteria, category_criteria, segment_criteria)

metrics_df[['geo_attribute', 'behavior_attribute', 'category_attribute', 'segment_attribute',
            'geo_criteria', 'behavior_criteria', 'category_criteria', 'segment_criteria']] = metrics_df.apply(assign_attributes_and_segment, axis=1, result_type='expand')

# Проценты атрибутов и сегментов
total_clients = len(metrics_df)
attribute_stats = {
    'geo': metrics_df['geo_attribute'].value_counts(normalize=True) * 100,
    'behavior': metrics_df['behavior_attribute'].value_counts(normalize=True) * 100,
    'category': metrics_df['category_attribute'].value_counts(normalize=True) * 100,
    'segment': metrics_df['segment_attribute'].value_counts(normalize=True) * 100
}
attribute_stats_df = pd.DataFrame({
    'Attribute': ['geo_' + k for k in attribute_stats['geo'].index] + 
                 ['behavior_' + k for k in attribute_stats['behavior'].index] + 
                 ['category_' + k for k in attribute_stats['category'].index] + 
                 ['segment_' + k for k in attribute_stats['segment'].index],
    'Percent': list(attribute_stats['geo'].values) + 
               list(attribute_stats['behavior'].values) + 
               list(attribute_stats['category'].values) + 
               list(attribute_stats['segment'].values)
})
attribute_stats_df.to_csv('/content/attribute_stats.csv', index=False)
print("Процентное распределение атрибутов и сегментов:")
display(attribute_stats_df)

conditions_data = [
    {"Тип": "Geo Attribute", "Название": "Национальный", "Условие": "acquirer_country_iso_share == 1.0"},
    {"Тип": "Geo Attribute", "Название": "Интернациональный", "Условие": "foreign_share_365d >= 0.01"},
    {"Тип": "Geo Attribute", "Название": "Местный", "Условие": "merchant_city_unique == 1"},
    {"Тип": "Geo Attribute", "Название": "Неопределённый", "Условие": "Не соответствует критериям geo"},
    {"Тип": "Behavior Attribute", "Название": "Рискованный", "Условие": "mcc_share_gaming > 0 or p2p_share > 0.3 (данные о кредитах отсутствуют)"},
    {"Тип": "Behavior Attribute", "Название": "Звездный", "Условие": "recency_days <= 30 and delta_avg_check_30d >= 0 and p2p_share == 0 (данные о кредитах отсутствуют)"},
    {"Тип": "Behavior Attribute", "Название": "Стандартный", "Условие": "Не соответствует критериям behavior"},
    {"Тип": "Behavior Attribute", "Название": "Неопределённый", "Условие": "Ошибка или недостаток данных"},
    {"Тип": "Category Attribute", "Название": "Любитель путешествий", "Условие": "mcc_share_travel >= 0.3"},
    {"Тип": "Category Attribute", "Название": "Гастроном", "Условие": "mcc_share_entertainment >= 0.3"},
    {"Тип": "Category Attribute", "Название": "Модник", "Условие": "mcc_share_fashion >= 0.3"},
    {"Тип": "Category Attribute", "Название": "Домосед", "Условие": "mcc_share_home >= 0.3"},
    {"Тип": "Category Attribute", "Название": "Техногик", "Условие": "mcc_share_tech >= 0.3"},
    {"Тип": "Category Attribute", "Название": "Геймер", "Условие": "mcc_share_gaming >= 0.3"},
    {"Тип": "Category Attribute", "Название": "Стандартный", "Условие": "Все mcc_share < 0.3"},
    {"Тип": "Category Attribute", "Название": "Неопределённый", "Условие": "Ошибка или недостаток данных"},
    {"Тип": "Segment Attribute", "Название": "Понижающийся", "Условие": "avg_check_0_30 < avg_check_30_60 and avg_check_30_60 > 0"},
    {"Тип": "Segment Attribute", "Название": "Повышающийся", "Условие": "sum_0_30 > sum_30_60"},
    {"Тип": "Segment Attribute", "Название": "Стабильный", "Условие": "abs(sum_0_30 - sum_30_60) / sum_30_60 < 0.1 and sum_30_60 > 0"},
    {"Тип": "Segment Attribute", "Название": "Остановившийся", "Условие": "avg_check_0_30 == 0"},
    {"Тип": "Segment Attribute", "Название": "Неопределённый", "Условие": "Не соответствует критериям сегмента"}
]

conditions_df = pd.DataFrame(conditions_data)
conditions_df.to_csv("/content/conditions.csv", index=False)
print("Таблица условий сохранена как /content/conditions.csv")
display(conditions_df)

print("\nПервые 5 строк с атрибутами и сегментами:")
display(metrics_df[['card_id', 'geo_attribute', 'behavior_attribute', 'category_attribute', 'segment_attribute']].head())

# %% [markdown]
# ## 5. Стандартизация фичей

numeric_cols = [
    'days_since_first_tx', 'recency_days', 'frequency_90d', 'monetary_90d_kzt', 'average_check_kzt',
    'delta_count_30d', 'delta_sum_30d', 'delta_avg_check_30d', 'mcc_share_travel', 'mcc_share_entertainment',
    'mcc_share_fashion', 'mcc_share_home', 'mcc_share_tech', 'mcc_share_gaming', 'p2p_share',
    'acquirer_country_iso_share', 'merchant_city_unique', 'foreign_share_365d'
]

try:
    features = metrics_df[numeric_cols].fillna(metrics_df[numeric_cols].median())
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(features)
    joblib.dump(scaler, "/content/scaler.pkl")
    print("StandardScaler сохранён как /content/scaler.pkl")
except Exception as e:
    print(f"Ошибка при стандартизации фичей: {e}")
    raise

# %% [markdown]
# ## 6. Подбор числа компонент для GMM

bic_scores = []
silhouette_scores = []
k_range = range(2, 11)

for k in k_range:
    try:
        gmm = GaussianMixture(n_components=k, covariance_type='full', init_params='kmeans', max_iter=300, random_state=42)
        gmm.fit(X_scaled)
        bic_scores.append(gmm.bic(X_scaled))
        labels = gmm.predict(X_scaled)
        score = silhouette_score(X_scaled, labels)
        silhouette_scores.append(score)
    except Exception as e:
        logging.error(f"Ошибка при GMM для k={k}: {e}")
        continue

plt.figure(figsize=(10, 5))
plt.plot(k_range, bic_scores, "bo-")
plt.xlabel("Число компонент (k)")
plt.ylabel("BIC")
plt.title("BIC для выбора k в GMM")
plt.savefig("/content/bic_plot.png")
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(k_range, silhouette_scores, "bo-")
plt.xlabel("Число компонент (k)")
plt.ylabel("Silhouette Score")
plt.title("Silhouette Score для выбора k в GMM")
plt.savefig("/content/silhouette_plot.png")
plt.show()

# %% [markdown]
# ## 7. GMM кластеризация

optimal_k = 7
try:
    gmm = GaussianMixture(n_components=optimal_k, covariance_type='full', init_params='kmeans', max_iter=300, random_state=42)
    metrics_df['gmm_cluster'] = gmm.fit_predict(X_scaled)

    silhouette = silhouette_score(X_scaled, metrics_df['gmm_cluster'])
    print(f"BIC: {gmm.bic(X_scaled):.2f}")
    print(f"Silhouette Score: {silhouette:.2f}")
    logging.info(f"BIC: {gmm.bic(X_scaled):.2f}, Silhouette Score: {silhouette:.2f}")

    initial_types = [("input", FloatTensorType([None, X_scaled.shape[1]]))]
    onnx_model = convert_sklearn(gmm, initial_types=initial_types, target_opset=11)
    onnx.save_model(onnx_model, "/content/client_segmentation.onnx")
    print("Модель экспортирована в /content/client_segmentation.onnx")
except Exception as e:
    print(f"Ошибка при кластеризации: {e}")
    raise

# %% [markdown]
# ## 8. Анализ кластеров и сегментов

try:
    cluster_stats = metrics_df.groupby('gmm_cluster').agg({
        'card_id': 'count',
        **{col: 'mean' for col in numeric_cols},
        'geo_attribute': lambda x: x.mode()[0] if not x.empty else "Неизвестно",
        'behavior_attribute': lambda x: x.mode()[0] if not x.empty else "Неизвестно",
        'category_attribute': lambda x: x.mode()[0] if not x.empty else "Неизвестно",
        'segment_attribute': lambda x: x.mode()[0] if not x.empty else "Неизвестно",
        'issuer_bank_name': lambda x: x.mode()[0] if not x.empty else "Неизвестно",
        'client_name': lambda x: x.iloc[0] if not x.empty else "Неизвестно"
    }).reset_index()

    cluster_stats['percent'] = cluster_stats['card_id'] / cluster_stats['card_id'].sum() * 100
    cluster_stats = cluster_stats.rename(columns={'card_id': 'client_count'})
    print("Характеристики кластеров:")
    display(cluster_stats.round(2))

    cluster_stats.to_csv("/content/cluster_stats.csv", index=False)
except Exception as e:
    print(f"Ошибка при анализе кластеров: {e}")
    raise

# %% [markdown]
# ## 9. Визуализация кластеров

try:
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=X_scaled[:, numeric_cols.index('recency_days')], 
                    y=X_scaled[:, numeric_cols.index('monetary_90d_kzt')], 
                    hue=metrics_df['gmm_cluster'], palette="deep")
    plt.xlabel('recency_days (scaled)')
    plt.ylabel('monetary_90d_kzt (scaled)')
    plt.title("Визуализация GMM кластеров")
    plt.savefig("/content/cluster_visualization.png")
    plt.show()
except Exception as e:
    print(f"Ошибка при визуализации кластеров: {e}")
    raise

# %% [markdown]
# ## 10. Data Dictionary

data_dict = [
    {"Название поля": "card_id", "Тип данных": "Int64", "Формула/Источник": "Уникальный идентификатор клиента/карты."},
    {"Название поля": "transaction_timestamp", "Тип данных": "datetime64[us]", "Формула/Источник": "Временная метка транзакции."},
    {"Название поля": "issuer_bank_name", "Тип данных": "object", "Формула/Источник": "Название банка-эмитента."},
    {"Название поля": "merchant_mcc", "Тип данных": "Int64", "Формула/Источник": "Код категории мерчанта (MCC)."},
    {"Название поля": "merchant_city", "Тип данных": "string", "Формула/Источник": "Город мерчанта."},
    {"Название поля": "transaction_type", "Тип данных": "string", "Формула/Источник": "Тип транзакции (например, 'POS', 'WEB', 'P2P')."},
    {"Название поля": "transaction_amount_kzt", "Тип данных": "Float64", "Формула/Источник": "Сумма транзакции в тенге (KZT)."},
    {"Название поля": "acquirer_country_iso", "Тип данных": "string", "Формула/Источник": "Код страны эквайера (например, 'KAZ')."},
    {"Название поля": "wallet_type", "Тип данных": "string", "Формула/Источник": "Тип кошелька (например, 'Apple Pay')."},
    {"Название поля": "days_since_first_tx", "Тип данных": "int64", "Формула/Источник": "(current_date - MIN(transaction_timestamp)).days"},
    {"Название поля": "recency_days", "Тип данных": "int64", "Формула/Источник": "(current_date - MAX(transaction_timestamp)).days"},
    {"Название поля": "frequency_90d", "Тип данных": "int64", "Формула/Источник": "COUNT(transaction_id WHERE transaction_timestamp >= current_date - 90 days), 0 если нет транзакций"},
    {"Название поля": "monetary_90d_kzt", "Тип данных": "Float64", "Формула/Источник": "SUM(transaction_amount_kzt WHERE transaction_timestamp >= current_date - 90 days), 0 если нет транзакций"},
    {"Название поля": "average_check_kzt", "Тип данных": "Float64", "Формула/Источник": "monetary_90d_kzt / frequency_90d, 0 если frequency_90d = 0"},
    {"Название поля": "delta_count_30d", "Тип данных": "int64", "Формула/Источник": "COUNT(0–30 days) - COUNT(30–60 days)"},
    {"Название поля": "delta_sum_30d", "Тип данных": "Float64", "Формула/Источник": "SUM(0–30 days) - SUM(30–60 days), 0 если нет транзакций"},
    {"Название поля": "delta_avg_check_30d", "Тип данных": "Float64", "Формула/Источник": "AVG(0–30 days) - AVG(30–60 days), 0 если нет транзакций"},
    {"Название поля": "sum_0_30", "Тип данных": "Float64", "Формула/Источник": "SUM(transaction_amount_kzt WHERE transaction_timestamp >= current_date - 30 days), 0 если нет транзакций"},
    {"Название поля": "sum_30_60", "Тип данных": "Float64", "Формула/Источник": "SUM(transaction_amount_kzt WHERE transaction_timestamp BETWEEN current_date - 60 days AND current_date - 30 days), 0 если нет транзакций"},
    {"Название поля": "avg_check_0_30", "Тип данных": "Float64", "Формула/Источник": "sum_0_30 / COUNT(0–30 days), 0 если нет транзакций"},
    {"Название поля": "avg_check_30_60", "Тип данных": "Float64", "Формула/Источник": "sum_30_60 / COUNT(30–60 days), 0 если нет транзакций"},
    {"Название поля": "mcc_share_travel", "Тип данных": "Float64", "Формула/Источник": "SUM(MCC 3000–3299) / SUM(transaction_amount_kzt), 0 если нет транзакций"},
    {"Название поля": "mcc_share_entertainment", "Тип данных": "Float64", "Формула/Источник": "SUM(MCC 5812, 5814) / SUM(transaction_amount_kzt), 0 если нет транзакций"},
    {"Название поля": "mcc_share_fashion", "Тип данных": "Float64", "Формула/Источник": "SUM(MCC 5611, 5621, 5631, 5641, 5651, 5661, 5691, 5699) / SUM(transaction_amount_kzt), 0 если нет транзакций"},
    {"Название поля": "mcc_share_home", "Тип данных": "Float64", "Формула/Источник": "SUM(MCC 5200, 5211, 5231, 5251) / SUM(transaction_amount_kzt), 0 если нет транзакций"},
    {"Название поля": "mcc_share_tech", "Тип данных": "Float64", "Формула/Источник": "SUM(MCC 5732–5734) / SUM(transaction_amount_kzt), 0 если нет транзакций"},
    {"Название поля": "mcc_share_gaming", "Тип данных": "Float64", "Формула/Источник": "SUM(MCC 7994) / SUM(transaction_amount_kzt), 0 если нет транзакций"},
    {"Название поля": "p2p_share", "Тип данных": "Float64", "Формула/Источник": "COUNT(transaction_type = 'P2P') / COUNT(*), 0 если нет транзакций"},
    {"Название поля": "acquirer_country_iso_share", "Тип данных": "Float64", "Формула/Источник": "COUNT(acquirer_country_iso = 'KAZ') / COUNT(*), 0 если нет транзакций"},
    {"Название поля": "merchant_city_unique", "Тип данных": "int64", "Формула/Источник": "COUNT(DISTINCT merchant_city)"},
    {"Название поля": "foreign_share_365d", "Тип данных": "Float64", "Формула/Источник": "SUM(transaction_amount_kzt WHERE acquirer_country_iso != 'KAZ' AND transaction_timestamp >= current_date - 365 days) / SUM(transaction_amount_kzt), 0 если нет транзакций"},
    {"Название поля": "gmm_cluster", "Тип данных": "int32", "Формула/Источник": "Номер кластера (0–6) из GMM"},
    {"Название поля": "geo_attribute", "Тип данных": "string", "Формула/Источник": "Национальный (acquirer_country_iso_share == 1.0), Интернациональный (foreign_share_365d >= 0.01), Местный (merchant_city_unique == 1), Неопределённый (иначе)"},
    {"Название поля": "behavior_attribute", "Тип данных": "string", "Формула/Источник": "Рискованный (mcc_share_gaming > 0 or p2p_share > 0.3), Звездный (recency_days <= 30 and delta_avg_check_30d >= 0 and p2p_share == 0), Стандартный (иначе), Неопределённый (ошибка)"},
    {"Название поля": "category_attribute", "Тип данных": "string", "Формула/Источник": "Любитель путешествий (mcc_share_travel >= 0.3), Гастроном (mcc_share_entertainment >= 0.3), Модник (mcc_share_fashion >= 0.3), Домосед (mcc_share_home >= 0.3), Техногик (mcc_share_tech >= 0.3), Геймер (mcc_share_gaming >= 0.3), Стандартный (все mcc_share < 0.3), Неопределённый (ошибка)"},
    {"Название поля": "segment_attribute", "Тип данных": "string", "Формула/Источник": "Понижающийся (avg_check_0_30 < avg_check_30_60 and avg_check_30_60 > 0), Повышающийся (sum_0_30 > sum_30_60), Стабильный (abs(sum_0_30 - sum_30_60) / sum_30_60 < 0.1 and sum_30_60 > 0), Остановившийся (avg_check_0_30 == 0), Неопределённый (иначе)"},
    {"Название поля": "geo_criteria", "Тип данных": "object", "Формула/Источник": "Критерии присвоения geo_attribute"},
    {"Название поля": "behavior_criteria", "Тип данных": "object", "Формула/Источник": "Критерии присвоения behavior_attribute"},
    {"Название поля": "category_criteria", "Тип данных": "object", "Формула/Источник": "Критерии присвоения category_attribute"},
    {"Название поля": "segment_criteria", "Тип данных": "object", "Формула/Источник": "Критерии присвоения segment_attribute"},
    {"Название поля": "client_name", "Тип данных": "string", "Формула/Источник": "Сгенерированное ФИО (Faker)"},
    {"Название поля": "attribute_percent", "Тип данных": "Float64", "Формула/Источник": "Процент клиентов с данным атрибутом/сегментом: client_count / total_clients * 100 (см. attribute_stats.csv)"}
]

try:
    data_dict_df = pd.DataFrame(data_dict)
    data_dict_df.to_csv("/content/data_dictionary.csv", index=False, encoding='utf-8')
    print("Data Dictionary сохранён как /content/data_dictionary.csv")
    display(data_dict_df)
except Exception as e:
    print(f"Ошибка при создании Data Dictionary: {e}")
    raise

# %% [markdown]
# ## 11. Результаты сегментации

try:
    result_df = metrics_df[['card_id', 'gmm_cluster', 'geo_attribute', 'behavior_attribute', 
                           'category_attribute', 'segment_attribute', 'geo_criteria', 
                           'behavior_criteria', 'category_criteria', 'segment_criteria',
                           'client_name', 'issuer_bank_name']]
    result_df.to_parquet("/content/segmented.parquet", index=False)
    print("Результаты сегментации сохранены как /content/segmented.parquet")
    print("\nПервые 5 строк результатов:")
    display(result_df.head())
except Exception as e:
    print(f"Ошибка при сохранении результатов: {e}")
    raise